Version 2023.12.29 - A. Lundervold

[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MMIV-ML/ELMED219/blob/main/Lab0-ML/Lab0-GT-and-complex-network-analysis/use-case-large-music-pref-network/1-network-of-music-pref.ipynb)

# 1. Practical Session: Networks of music preferences 

#### Using the [Last.fm](http://millionsongdataset.com/) (millionsong) dataset



For this activity, we will create graphs using [NetworkX](https://networkx.org/) and traditional libraries for scientific computing such as [Pandas](https://pandas.pydata.org/) and [NumPy](https://numpy.org/) (data transformation and processing) plus [Matplotlib](https://matplotlib.org/) (data visualization).


## 1.1. About the dataset


We will use a reduced version of the Last.fm dataset publicly available from works `[1-4]`. This dataset contains the recordings of 1000 users from Feb. 2005 until Jun. 2009. This dataset contains the following columns:

+ `userid`: User ID (anonymized)
+ `timestamp`: Timestamp of the record. The format is `YYYY-MM-DD` followed by `hh:mm:ss`. Example: `2009-04-08T01:57:47Z`.
+ `musicbrainz-artist-id`: Artists ID.
+ `artist-name`: Artist name.
+ `musicbarinz-track-id`: Track ID.
+ `track-name`: Track name.

For this activity, we will use the `userid`, `timestamp`, `artist-id`, and `artist-name`.

---

### Check your platform

In [7]:
import platform
import os
from os.path import expanduser
home = expanduser("~")

if platform.system() == 'Darwin':
    print(f'OK, you are running on MacOS ({platform.version()})')
    OVERLEAF_dir = f'{home}/Dropbox/Apps/Overleaf/My_project'
    TREE = '/usr/local/bin/tree -L 1'          # MacOS (brew install tree)
    MacOS = True
    Linux = False
if platform.system() == 'Linux':
    print(f'OK, you are running on Linux ({platform.version()})')
    OVERLEAF_dir = f'{home}/Dropbox/Apps/Overleaf/My_project'
    TREE = '/usr/bin/tree -L 4'
    Linux = True
    MacOS = False
if platform.system() == 'Windows':
    print(f'OK, but consider to install WSL for Windows10 since you are running on {platform.system()}')
    print('Check https://docs.microsoft.com/en-us/windows/wsl/install-win10')
    MacOS = False
    Linux = False

OK, you are running on MacOS (Darwin Kernel Version 23.2.0: Wed Nov 15 21:54:10 PST 2023; root:xnu-10002.61.3~2/RELEASE_X86_64)


### Check if Colab

In [8]:
# This is a quick check of whether the notebook is currently running on Google Colaboratory
# as that makes some difference for the code below.
# We'll do this in every notebook of the course.

try:
    import google.colab
    # If this statement executes without error, you're in a Colab environment.
    is_colab = True
    print("Running in Google Colab.")
except ImportError:
    # An ImportError means you're not in a Colab environment.
    is_colab = False
    print("Not running in Google Colab.")

Not running in Google Colab.


## 1.2. Objectives and structure of practical session 

### 1.2.1. Objectives

1. Getting to know how to create, visualize, and analyse networks using NetworkX.
2. Apply widely used topology and centrality metrics.
3. Define networks across time for longitudinal analyses. 

### 1.2.2. Structure

**Part I : Data Exploration.**

Explore, clean, transform, and visualize data for  further analyses.



**Part II : Analyses of network of artists.**

Create, visualize, and analyze a binary Graph $G(V,E)$ such that $V$ correspond to a set of artists found in this dataset (i.e., vertices/nodes). Two artists $v_i$ and $v_j$ are connected by an edge $e_{ij}=(n_i, n_j) \in E$ if a user listen to both of them in a given time $t \in T$.

**Part III : Analyses of evolving networks based on user's preferences.**

Create, visualize, and analyze *weighted graphs* $G_t(V, E_t, W_t)$ for every month $t \in T$ in our dataset. The set of nodes $V$ corresponds to a set of artists from the entire dataset. Two artists $v_{t, i}$ and $v_{t, j}$ are connected by an edge $e_{t,ij}=(n_{t,i}, n_{t,j}) \in E$ if a user listen to both of them in a given month $t$. Such edges contains a weight $w_{t,ij} \in W_t$ corresponds to the amount of users that connect both artists $v_{t,i}$ and $v_{t,j}$.

Answer to the following questions:
+ Is the network topology stable?
+ For this group of users, which is the most central artist? Is this most central artist the same if we use other centrality measures?
+ Are these centrality measures stable across time?

---

## 1.3. Playground

### 1.3.1. Part 1: Data Exploration

**Required packages**

In [9]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np

**Reading dataset and data exploration**

In [10]:
if is_colab:
    ! wget https://raw.githubusercontent.com/bsotomayorg/UACh-MIN-Graph_NLP/main/dataset/DS_userid-playlist-record.tsv
    df_timestamp = pd.read_csv("DS_userid-playlist-record.tsv", sep='\t')
else:
    df_timestamp = pd.read_csv("../data/DS_userid-playlist-record.tsv", sep='\t')

In [11]:
df_timestamp

,userid,timestamp,musicbrainz-artist-id,artist-name,musicbrainz-track-id,track-name
0,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
1,user_000001,2009-05-01T08:24:50Z,a3934f47-c4cc-4e73-8b37-ce41775797d4,Lisa Shaw,NaN,Inside My Love (Extended Mix)
2,user_000001,2009-04-29T13:00:55Z,495fe320-091e-43eb-9321-54a20e7c3f98,Sneaky Sound System,NaN,Pictures [Radio Edit]/Radio Edit
3,user_000001,2009-04-26T14:56:08Z,495fe320-091e-43eb-9321-54a20e7c3f98,Sneaky Sound System,a15efe90-1a35-4633-8888-ab4bf879527a,Kansas City
4,user_000001,2009-04-24T09:58:21Z,NaN,Designed People,NaN,Radio 808 Mars
...,...,...,...,...,...,...
191504,user_001000,2008-02-08T07:20:43Z,b7ddce8b-9e5c-46bd-9d33-41b134ce1a7f,Wolf Parade,27dba9c5-27de-4c07-a7bd-f279cacc34d5,I'Ll Believe In Anything
191505,user_001000,2008-02-07T02:44:35Z,f96627af-7aac-4e16-9245-c5661eb47199,Muchy,30bc57a9-5646-49dd-8870-4cdca33a21f3,Terroromans
191506,user_001000,2008-02-03T19:40:37Z,e549424e-8a71-4257-acf7-18e798127cae,Katrah-Quey,1d55d748-0867-4c7f-b9ec-39fbb527af86,Double Ingrown Nail June
191507,user_001000,2008-01-31T21:50:28Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,8618e305-4289-4574-8acc-d5d8ec6b7411,House Of Cards


In [12]:
# Delete the file if it exists

if os.path.exists("DS_userid-playlist-record.tsv"):
    os.remove("DS_userid-playlist-record.tsv")

**Data cleaning**

Delete from the dataset such rows that contain `NaN`s.

Show the amount of unique artist names in the dataset

Show the amount of unique users

Show the amount of records (i.e., rows) per yearh and month (`YYYY-MM`) format. Use matlpotlib to create whether a histogram or scatter plot (i.e., x-axis contains the year-month and y-axis the count of records/rows.

---

### 1.3.2. Part 2: Creation of one single network


+ Definition of network based of artists based on user's preferences. To this purpose, use the data for only one month. Then, model the solution using a binary graph.


#### 1.3.2.1. Timestamp selection

Select only one month (e..g, Jan. 2007)

Get the amount of users and artists that are part of the records for the month you selected. This will be used to create the matrix of nodes/vertices and edges (i.e. $V \times E$).

#### 1.3.2.2. Definition of matrix $ V \times U$

Create a matrix of vertices and edges (i.e. $V \times U$) such that it stores the set of artists ($V$) that each user ($U$) listened to. 

Make sure this matrix contains only 0's and 1's as values.

In [ ]:
V = # number of artists
U = # number of users

print("V = %i; U = %i" % (V, U))

Populate matrix $V \times U$ (i.e. `mat_VU`)

Check your artists-users matrix by plotting it! 

You can use `plt.imshow(mat_VU, cmap='Greys_r')` if you want.

#### 1.3.2.3. Binary graph's adjacency matrix

We can define a routine to convert our artist-users matrix into a binary adjacency matrix $A$.

In order to do so, we might check whether two artists $v_i$ and $v_j$ were listened to by at least one user.

Make sure that the adjacency matrix contains only 0's and 1's! Also, consider that our binary matrix represents an undirected graph. Thus, $A$ is symmetric (and you can reduce the computations to $\frac{V \cdot (V-1)}{2}$ operations (Still $\mathcal{O}(V^2)$ tho).

In [ ]:
A = # ...

In [10]:
# plot it!

#### 1.3.2.4. Graph Visualization

Now, let's create a graph that contains the information have in our adjacency matrix $A$.

Using Networkx, we can create a network using the adjacency matrix as input. For example, to create a graph as `G_bin`, we do:

We can use a circular layout as first visualization of our network:

To many connections? let's try with the Spring layout:

What can we observe already? Is it a full graph? 

#### 1.3.2.5. Visualization of the largest (main) component

Some networks may be not fully-connected. In this sub-section we will focus our analyses only in the largest component of our graph.

In order to do so, we need to detect every connected components as sub-graphs:

In [ ]:
S_bin = [G_bin.subgraph(c).copy() for c in nx.connected_components(G_bin)]

In [ ]:
print("Number of nodes per component:")
np.array([ len(S_bin[i]) for i in range(len(S_bin))])

Let's visualize the largest components using circular and spring layouts:

In [ ]:
# computing layouts
pos = nx.layout.circular_layout(S_bin[0])
pos = nx.spring_layout(S_bin[0], pos=pos, k=0.35, iterations=100, seed=0)

In [ ]:
# plot it

**Note:** We are considering one month and the connection only needs one shared artist.

#### 1.3.2.6. Topology analysis of the largest component

How is the structure of our largest sub-graph? In this part, we need to compute some topology measures. Specifically, we will compute:
+ the network size, 
+ number of links, 
+ average degree, 
+ density, 
+ average clustering coefficient, 
+ average number of triangles, and 
+ average path length.

In [12]:
def get_topology_values(G):
  # input:
  # G: Networkx Graph
  # out:
  # dict (key: metric name, value: metric value)

  d_topology_vals = {
      "network_size" : G.number_of_nodes(),
      "number_of_links" : G.number_of_edges(),
      "avg_degree" : np.mean(list(dict(G.degree()).values())),
      "density" : nx.density(G),
      "accumulated_degeree" : np.sum(list(dict(G.degree()).values())),
      "avg_clustering_coef" : nx.average_clustering(G),
      "avg_triangles" : np.mean(list(nx.triangles(G).values())),
      "avg_path_length" : nx.average_shortest_path_length(G),
  }

  return d_topology_vals

#### 1.3.2.7. Centrality analysis of the largest connected component

In this case, let's analyse the centrality of each node from only the largest connected component. We will use four centrality measures:
1. Degree centrality (`nx.degree_centrality()`)
2. Betweenness centrality (`nx.betweenness_centrality()`)
3. Closeness centrality (`nx.closeness_centrality()`)
4. Eigen-vector centrality (`nx.eigenvector_centrality()`)

Use the attribute `node_color` of  `nx.draw_networkx_nodes` to color each node based on its centrality value!

#### 1.3.2.8. Comparison of centrality distributions

#### 1.3.2.9. Questions

1. Which are the set of artists that connect the largest amount of users?
2. Which are the sets of artitsts that connect the top 10 artists more importants for connecting users? How to interpret each centrality measure in this case?

Depending in the centrality measure we are using we will get different results. Here, the "importance" of each artists have changed across metrics. Therefore, we need to consider the definition of each measure depending on what we want to measure from the network. Thus, this will change depending on the problem domain.

In [13]:
from functools import reduce

In [ ]:
reduce(np.intersect1d, (top_artists_degree, top_artists_betweenness, top_artists_closeness, top_artists_eigenvector))

**Note:** Remember, when you scale your data, you are changing the *range* of these values. When you normalize your data, you are changing the *shape of the distribution* of your data.


Good! Now that we have some information about the structure of our network, let's study how this evolves across time (i.e. months).

----

### 1.3.3. Part 3: Complex network analysis across months
	

In this section, we want to answer the following questions:
+ Study network of artists interconnected by LastFM user's preferences per month. Is the network of artist constantly changing? What are the features (i.e., topology measures) of the each network? Do these result change if we use weighted graphs instead of binary ones?
+ Which is the most central artist per month? Is it the same one? Is it the same if we use another centrality measure?
+ Which are the most top 3 similar artists of most central one?

Are these artists connected always in the same way? How their relation change across months? Using our LastFM dataset, we can check that by creating networks per month. We can look at the different topology measures and see whether they remain the same or fluctuate.

#### 1.3.3.1. Timestamp selection and filtering

For computational costs, let's consider only the first TOP-50 artists for our analyses (i.e., `max_V = 50`).


#### 1.3.3.2 Definition of matrix $V \times U \times T$

#### 1.3.3.3. Weighted graph's adjacency matrix per month

#### 1.3.3.4. Graph visualizations

Visualization pf graphs and adjacency matrices across time

#### 1.3.3.5. Analysis across time: Changes in networks' topology 

#### 1.3.3.6. Visualization of changes of networks structure

#### 1.3.3.7. Centrality analysis across time

Top influencial bands for users from LastFM comunnity:

---

# References

[1] Benson, A. R., Kumar, R., & Tomkins, A. (2018, February). A discrete choice model for subset selection. In Proceedings of the eleventh ACM international conference on web search and data mining (pp. 37-45).

[2] Celma, O. (2010). Music recommendation and discovery: The long tail, long fail, and long play in the digital music space. Springer Science & Business Media.

[3] Lamere, P. The LastFM ArtistTags2007 Data set, 2008. URL http://musicmachinery.com/2010/11/10/lastfm-artisttags2007.

[4] Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere.  The Million Song Dataset. In Proceedings of the 12th International Society
for Music Information Retrieval Conference (ISMIR 2011), 2011. A [[tutorial](http://millionsongdataset.com/sites/default/files/tutorial1.py.txt)] is available as well.
